In [50]:
import xml.etree.ElementTree as etree
import random, math
import numpy as np
import scipy as sp
import scipy.stats as stats
import matplotlib.pyplot as plt
#import simpy
import scipy.optimize as op
import time
import pymc3 as pm 
import pandas as pd
import theano.tensor as tt
import seaborn as sb

In [51]:
SEED = [20100425, 20100234]


data_ids = []
tester_ids = []
level_ids = []
contexts = []
left_obstacles = []
right_obstacles = []
start_lane = []
decisions = []
gender_ids = []

j = 0

In [52]:
for i in range(0,210):
    tree = etree.parse('log'+ str(i) + '.xml')
    root = tree.getroot()
    for child in root:
        if child.find('stagetype') == None:
            continue
        elif child.find('stagetype').text == 'trial':
            data_ids.append(j)
            tester_ids.append(i)
            level_ids.append(child.find('trialdescription').text)
            contexts.append(child.find('level').text)
            left_obstacles.append(child.find('leftobstacles').text)
            right_obstacles.append(child.find('rightobstacles').text)
            start_lane.append(child.find('startlane').text)
            decisions.append(child.find('answer').text)
            j += 1
        elif child.find('stagetype').text == 'q01':
            if child.find('displaytext-en').text == 'Please indicate your sex.':
                gender_ids.append(i)


In [53]:
data = []

print(len(data_ids))
x = 0
for i in range(0,210):
    data.append([])

for i in range(0,4942):
    
    print(contexts[i])
    print(left_obstacles[i])
    print(right_obstacles[i])
    print("###############")
    
    left_self_value = 0
    left_old_value = 0
    left_young_value = 0
    left_man_value = 0
    left_kneeling_value = 0
    left_pavement_value = 0
    
    right_self_value = 0
    right_old_value = 0
    right_young_value = 0
    right_man_value = 0
    right_kneeling_value = 0
    right_pavement_value = 0
    
    switch_self_value = 0
    switch_old_value = 0
    switch_young_value = 0
    switch_man_value = 0
    switch_kneeling_value = 0
    switch_pavement_value = 0
    
    stay_self_value = 0
    stay_old_value = 0
    stay_young_value = 0
    stay_man_value = 0
    stay_kneeling_value = 0
    stay_pavement_value = 0
    
    
    choice = 0 
    result = []
    if contexts[i] == "mountain":
        print("in mountain")
        ### left obstacle:
        
        man_var = left_obstacles[i].split('n')
        if man_var[1] == ",":
            left_self_value = 0
            left_old_value = 0
            left_young_value = 0
            left_man_value = 1
            left_kneeling_value = 0
            left_pavement_value = 0
        else:
            man_number = man_var[1][0]
            #left = [0,0,0,int(man_number),0,0]
            left_self_value = 0
            left_old_value = 0
            left_young_value = 0
            left_man_value = int(man_number)
            left_kneeling_value = 0
            left_pavement_value = 0
        level = level_ids[i].split('.')
        if level[0] == '2':
            #right = [1,0,0,0,0,0]
            print("in self")
            right_self_value = 1

        else:
            man_var = right_obstacles[i].split('n')
            if man_var[1] == ",":
                #right_man_value = [0,0,0,1,0,0]
                right_man_value = 1
            else:
                man_number = man_var[1][0]
                #right = [0,0,0,int(man_number),0,0]
                right_man_value = int(man_number)


    elif contexts[i] == "cityr":
        print("in cityr")
        man_boy = left_obstacles[i][0]
        if man_boy == "b":
            boy_var = left_obstacles[i].split('y')
            if boy_var[1] == ",":
                #left = [0,0,1,0,0,0]
                left_young_value = 1
            else:
                boy_number = boy_var[1][0]
                #left = [0,0,int(boy_number),0,0,0]
                left_young_value = int(boy_number)
        else:
            man_var = left_obstacles[i].split('n')
            if man_var[1] == ",":
                #left = [0,0,0,1,0,0]
                left_man_value = 1
            else:
                man_number = man_var[1][0]
                #left = [0,0,0,int(man_number),0,0]
                #left = [0,0,0,1,0,0]
                left_man_value = int(man_number)

        man_var = right_obstacles[i].split('n')
        if man_var[1] == ",":
            #right = [0,0,0,1,0,1]
            right_man_value = 1
            right_pavement_value = 1
        else:
            man_number = man_var[1][0]
            #right = [0,0,0,1,0,1]
            right_man_value = int(man_number)
            right_pavement_value = 1


    elif contexts[i] == "cityl":
        print("in cityl")
        man_boy = right_obstacles[i][0]
        if man_boy == "b":
            boy_var = right_obstacles[i].split('y')
            if boy_var[1] == ",":
                #right = [0,0,1,0,0,0]
                right_young_value = 1
            else:
                boy_number = boy_var[1][0]
                #right = [0,0,int(boy_number),0,0,0]
                #right = [0,0,1,0,0,0]
                right_young_value = int(boy_number)

        else:
            man_var = right_obstacles[i].split('n')
            if man_var[1] == ",":
                #right = [0,0,0,1,0,0]
                right_pavement_value = 1
            else:
                man_number = man_var[1][0]
                #right = [0,0,0,int(man_number),0,0]
                #right = [0,0,0,1,0,0]
                right_man_value = int(man_number)

        man_var = left_obstacles[i].split('n')
        if man_var[1] == ",":
            #left = [0,0,0,1,0,1]
            left_man_value = 1
            left_pavement_value = 1

        else:
            man_number = man_var[1][0]
            #left = [0,0,0,int(man_number),0,1]	
            #left = [0,0,0,1,0,1]
            left_man_value = int(man_number)
            left_pavement_value = 1	
    else:
        print("in suburb")
        if left_obstacles[i] == "boy,":
            #left = [0,0,1,0,0,0]
            left_young_value = 1
        elif left_obstacles[i] == "oldman,":
            #eft = [0,1,0,0,0,0]
            left_old_value = 1
        elif left_obstacles[i] == "kneelingman,":
            #eft = [0,0,0,0,1,0]
            left_kneeling_value = 1
        elif left_obstacles[i] == "man,":
            #left = [0,0,0,1,0,0]
            left_man_value = 1

        if right_obstacles[i] == "boy,":
            #right = [0,0,1,0,0,0]
            right_young_value = 1
        elif right_obstacles[i] == "oldman,":
            #right = [0,1,0,0,0,0]
            right_old_value = 1
        elif right_obstacles[i] == "kneelingman,":
            #right = [0,0,0,0,1,0]
            right_kneeling_value = 1
        elif right_obstacles[i] == "man,":
            #right = [0,0,0,1,0,0]
            right_man_value = 1


    ###### switch = 1 stay = 0      
    if(decisions[i] == "right"):
        if start_lane[i] == "left":
            choice = 1
            
            switch_self_value = right_self_value
            switch_old_value = right_old_value
            switch_young_value = right_young_value
            switch_man_value = right_man_value
            switch_kneeling_value = right_kneeling_value
            switch_pavement_value = right_pavement_value
            
            stay_self_value = left_self_value
            stay_old_value = left_old_value
            stay_young_value = left_young_value
            stay_man_value = left_man_value
            stay_kneeling_value = left_kneeling_value
            stay_pavement_value = left_pavement_value
        else:
            choice = 0
            
                
            switch_self_value = left_self_value
            switch_old_value = left_old_value
            switch_young_value = left_young_value
            switch_man_value = left_man_value
            switch_kneeling_value = left_kneeling_value
            switch_pavement_value = left_pavement_value
            
            stay_self_value = right_self_value
            stay_old_value = right_old_value
            stay_young_value = right_young_value
            stay_man_value = right_man_value
            stay_kneeling_value = right_kneeling_value
            stay_pavement_value = right_pavement_value
            
            
    else:
        if start_lane[i] == "left":
            choice = 0
            
            switch_self_value = right_self_value
            switch_old_value = right_old_value
            switch_young_value = right_young_value
            switch_man_value = right_man_value
            switch_kneeling_value = right_kneeling_value
            switch_pavement_value = right_pavement_value
            
            stay_self_value = left_self_value
            stay_old_value = left_old_value
            stay_young_value = left_young_value
            stay_man_value = left_man_value
            stay_kneeling_value = left_kneeling_value
            stay_pavement_value = left_pavement_value
        else:
            choice = 1
            
            switch_old_value = left_old_value
            switch_self_value = left_self_value
            switch_young_value = left_young_value
            switch_man_value = left_man_value
            switch_kneeling_value = left_kneeling_value
            switch_pavement_value = left_pavement_value
            
            stay_self_value = right_self_value
            stay_old_value = right_old_value
            stay_young_value = right_young_value
            stay_man_value = right_man_value
            stay_kneeling_value = right_kneeling_value
            stay_pavement_value = right_pavement_value

    """result.append(data_ids[i])
    result.append(tester_ids[i])
    result.append(left)
    result.append(right)
    result.append(choice)"""
    
    if tester_ids[i] in gender_ids:
        
        if tester_ids[i] == 0:
            print("hi")
        
        elif tester_ids[i - 1] != tester_ids[i]:
            x = x + 1
        


    

        result.append(x)

        #stay
        result.append(stay_self_value)
        result.append(stay_old_value)
        result.append(stay_young_value)
        result.append(stay_man_value)
        result.append(stay_kneeling_value)
        result.append(stay_pavement_value)

        #switch
        result.append(switch_self_value)
        result.append(switch_old_value)
        result.append(switch_young_value)
        result.append(switch_man_value)
        result.append(switch_kneeling_value)
        result.append(switch_pavement_value)

        result.append(choice)


        print("###############")


        data[tester_ids[i]].append(result)

4942
suburb
boy,
kneelingman,
###############
in suburb
hi
###############
cityl
man,
man5,
###############
in cityl
hi
###############
cityr
man2,
man,
###############
in cityr
hi
###############
suburb
man,
kneelingman,
###############
in suburb
hi
###############
mountain
man6,
empty,
###############
in mountain
in self
hi
###############
mountain
man2,
empty,
###############
in mountain
in self
hi
###############
mountain
man4,
empty,
###############
in mountain
in self
hi
###############
mountain
man,
man6,
###############
in mountain
hi
###############
cityr
boy2,
man,
###############
in cityr
hi
###############
cityr
man3,
man,
###############
in cityr
hi
###############
cityl
man2,
boy,
###############
in cityl
hi
###############
cityr
man4,
man,
###############
in cityr
hi
###############
cityr
boy,
man,
###############
in cityr
hi
###############
mountain
man3,
empty,
###############
in mountain
in self
hi
###############
mountain
man7,
empty,
###############
in mountain
in s

in mountain
in self
###############
mountain
man4,
empty,
###############
in mountain
in self
###############
mountain
man5,
empty,
###############
in mountain
in self
###############
cityl
man,
man3,
###############
in cityl
###############
suburb
man,
boy,
###############
in suburb
###############
mountain
man2,
empty,
###############
in mountain
in self
###############
cityr
boy,
man,
###############
in cityr
###############
cityl
man,
man7,
###############
in cityl
###############
mountain
man,
man6,
###############
in mountain
###############
mountain
man7,
empty,
###############
in mountain
in self
###############
suburb
oldman,
kneelingman,
###############
in suburb
###############
suburb
kneelingman,
boy,
###############
in suburb
###############
cityl
man,
boy2,
###############
in cityl
###############
mountain
man2,
man,
###############
in mountain
###############
mountain
man3,
empty,
###############
in mountain
in self
###############
mountain
man5,
empty,
###############
i

man,
man2,
###############
in cityl
###############
mountain
man2,
man,
###############
in mountain
###############
cityr
man6,
man,
###############
in cityr
###############
suburb
boy,
kneelingman,
###############
in suburb
###############
mountain
man2,
empty,
###############
in mountain
in self
###############
mountain
man6,
man,
###############
in mountain
###############
suburb
man,
oldman,
###############
in suburb
###############
mountain
man4,
empty,
###############
in mountain
in self
###############
suburb
kneelingman,
oldman,
###############
in suburb
###############
cityl
man,
man4,
###############
in cityl
###############
cityl
man,
boy2,
###############
in cityl
###############
mountain
man5,
empty,
###############
in mountain
in self
###############
suburb
boy,
man,
###############
in suburb
###############
suburb
man,
boy,
###############
in suburb
###############
suburb
kneelingman,
man,
###############
in suburb
###############
cityl
man,
man5,
###############
in city

###############
in suburb
###############
mountain
man,
man2,
###############
in mountain
###############
suburb
boy,
man,
###############
in suburb
###############
mountain
man6,
empty,
###############
in mountain
in self
###############
suburb
kneelingman,
boy,
###############
in suburb
###############
mountain
man,
man6,
###############
in mountain
###############
cityr
man5,
man,
###############
in cityr
###############
cityl
man,
boy2,
###############
in cityl
###############
mountain
man4,
man,
###############
in mountain
###############
mountain
man2,
empty,
###############
in mountain
in self
###############
cityr
man3,
man,
###############
in cityr
###############
suburb
kneelingman,
man,
###############
in suburb
###############
cityl
man,
man2,
###############
in cityl
###############
mountain
man5,
empty,
###############
in mountain
in self
###############
cityr
man6,
man,
###############
in cityr
###############
cityr
boy,
man,
###############
in cityr
###############
moun

mountain
man2,
empty,
###############
in mountain
in self
###############
mountain
man6,
empty,
###############
in mountain
in self
###############
suburb
man,
kneelingman,
###############
in suburb
###############
mountain
man7,
empty,
###############
in mountain
in self
###############
cityr
man5,
man,
###############
in cityr
###############
suburb
oldman,
boy,
###############
in suburb
###############
cityl
man,
man3,
###############
in cityl
###############
cityr
boy,
man2,
###############
in cityr
###############
suburb
man,
boy,
###############
in suburb
###############
mountain
man6,
man,
###############
in mountain
###############
suburb
kneelingman,
boy,
###############
in suburb
###############
cityl
man,
man2,
###############
in cityl
###############
suburb
oldman,
kneelingman,
###############
in suburb
###############
mountain
man7,
empty,
###############
in mountain
in self
###############
mountain
man,
man4,
###############
in mountain
###############
mountain
man4,
empt

...

###############
cityl
man,
boy2,
###############
in cityl
###############
cityl
man,
man7,
###############
in cityl
###############
suburb
man,
oldman,
###############
in suburb
###############
cityl
man2,
boy,
###############
in cityl
###############
mountain
man,
man4,
###############
in mountain
###############
suburb
man,
kneelingman,
###############
in suburb
###############
mountain
man6,
empty,
###############
in mountain
in self
###############
cityr
man6,
man,
###############
in cityr
###############
cityl
man,
man2,
###############
in cityl
###############
cityr
boy,
man,
###############
in cityr
###############
mountain
man5,
empty,
###############
in mountain
in self
###############
mountain
man6,
man,
###############
in mountain
###############
mountain
man2,
empty,
###############
in mountain
in self
###############
mountain
man4,
empty,
###############
in mountain
in self
###############
cityl
man,
man7,
###############
in cityl
###############
mountain
man3,
empty,
####

in suburb
mountain
man2,
empty,
###############
in mountain
in self
cityr
man2,
man,
###############
in cityr
mountain
man,
man2,
###############
in mountain
mountain
man6,
empty,
###############
in mountain
in self
cityl
man,
man3,
###############
in cityl
mountain
man4,
man,
###############
in mountain
mountain
man6,
man,
###############
in mountain
###############
cityr
man3,
man,
###############
in cityr
###############
cityr
boy,
man,
###############
in cityr
###############
cityl
man,
man7,
###############
in cityl
###############
suburb
boy,
oldman,
###############
in suburb
###############
cityl
man,
man4,
###############
in cityl
###############
suburb
man,
oldman,
###############
in suburb
###############
suburb
kneelingman,
oldman,
###############
in suburb
###############
cityl
man,
man6,
###############
in cityl
###############
suburb
man,
boy,
###############
in suburb
###############
mountain
man2,
empty,
###############
in mountain
in self
###############
mountain
man6,

###############
cityr
boy,
man,
###############
in cityr
###############
cityl
man2,
boy,
###############
in cityl
###############
cityr
man2,
man,
###############
in cityr
###############
suburb
man,
kneelingman,
###############
in suburb
###############
mountain
man4,
man,
###############
in mountain
###############
suburb
boy,
kneelingman,
###############
in suburb
###############
cityl
man,
man5,
###############
in cityl
###############
cityr
boy,
man,
###############
in cityr
###############
suburb
oldman,
man,
###############
in suburb
###############
mountain
man,
man6,
###############
in mountain
###############
cityr
man6,
man,
###############
in cityr
###############
suburb
kneelingman,
oldman,
###############
in suburb
###############
suburb
boy,
oldman,
###############
in suburb
###############
suburb
kneelingman,
man,
###############
in suburb
###############
mountain
man5,
empty,
###############
in mountain
in self
###############
mountain
man4,
empty,
###############
in 

cityl
man,
boy2,
###############
in cityl
###############
suburb
oldman,
man,
###############
in suburb
###############
cityl
man,
man2,
###############
in cityl
###############
suburb
kneelingman,
man,
###############
in suburb
###############
mountain
man2,
man,
###############
in mountain
###############
mountain
man5,
empty,
###############
in mountain
in self
###############
cityr
man5,
man,
###############
in cityr
###############
cityr
man4,
man,
###############
in cityr
###############
mountain
man4,
empty,
###############
in mountain
in self
###############
cityr
boy,
man2,
###############
in cityr
###############
suburb
kneelingman,
boy,
###############
in suburb
###############
cityl
man,
man3,
###############
in cityl
###############
suburb
boy,
oldman,
###############
in suburb
###############
mountain
man7,
empty,
###############
in mountain
in self
###############
mountain
man3,
empty,
###############
in mountain
in self
###############
cityr
man6,
man,
###############
i

boy,
man2,
###############
in cityr
###############
cityr
man6,
man,
###############
in cityr
###############
mountain
man,
man4,
###############
in mountain
###############
suburb
boy,
kneelingman,
###############
in suburb
###############
cityl
man,
boy,
###############
in cityl
###############
mountain
man2,
empty,
###############
in mountain
in self
###############
cityr
boy2,
man,
###############
in cityr
###############
cityr
man7,
man,
###############
in cityr
###############
suburb
kneelingman,
oldman,
###############
in suburb
###############
cityl
man,
man6,
###############
in cityl
###############
suburb
kneelingman,
boy,
###############
in suburb
###############
mountain
man5,
empty,
###############
in mountain
in self
###############
suburb
man,
boy,
###############
in suburb
###############
mountain
man,
man2,
###############
in mountain
###############
mountain
man,
man4,
###############
in mountain
###############
suburb
oldman,
man,
###############
in suburb
##########

In [54]:
data_orig = []

for i in range(0,len(data)):
    if data[i] == []:
        print(i)
        continue
    else:
        data_orig.append([])

x = 0

for i in range(0,len(data)):
    if data[i] == []:
        continue
    else:
        for j in data[i]:            
            data_orig[x].append(j)
        x = x + 1
    
for j in data_orig:
    print(j)

20
25
34
38
39
42
55
57
164
[[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1], [0, 0, 0, 0, 5, 0, 0, 0, 0, 0, 1, 0, 1, 1], [0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 1, 0, 1, 1], [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0], [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 6, 0, 0, 1], [0, 0, 0, 0, 2, 0, 0, 1, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 4, 0, 0, 1, 0, 0, 0, 0, 0, 1], [0, 0, 0, 0, 6, 0, 0, 0, 0, 0, 1, 0, 0, 1], [0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1], [0, 0, 0, 0, 3, 0, 0, 0, 0, 0, 1, 0, 1, 1], [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 2, 0, 1, 1], [0, 0, 0, 0, 4, 0, 0, 0, 0, 0, 1, 0, 1, 1], [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1], [0, 0, 0, 0, 3, 0, 0, 1, 0, 0, 0, 0, 0, 1], [0, 0, 0, 0, 7, 0, 0, 1, 0, 0, 0, 0, 0, 1], [0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0], [0, 0, 0, 0, 6, 0, 0, 0, 0, 0, 1, 0, 1, 1], [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 2, 0, 0, 0], [0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0], [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0], [0, 0, 0, 0, 4, 0, 0, 0, 0, 0, 1, 0, 0, 1], [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 5, 0, 0, 0], [0,

[[44, 0, 0, 0, 5, 0, 0, 0, 0, 0, 1, 0, 1, 0], [44, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1], [44, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1], [44, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0], [44, 1, 0, 0, 0, 0, 0, 0, 0, 0, 5, 0, 0, 1], [44, 0, 0, 0, 2, 0, 0, 0, 0, 0, 1, 0, 0, 1], [44, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1], [44, 0, 0, 2, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1], [44, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1], [44, 0, 0, 0, 3, 0, 0, 0, 0, 0, 1, 0, 1, 0], [44, 0, 0, 0, 1, 0, 0, 0, 0, 0, 4, 0, 0, 0], [44, 1, 0, 0, 0, 0, 0, 0, 0, 0, 6, 0, 0, 1], [44, 0, 0, 0, 1, 0, 0, 0, 0, 0, 6, 0, 0, 0], [44, 0, 0, 0, 7, 0, 0, 0, 0, 0, 1, 0, 1, 0], [44, 0, 0, 0, 4, 0, 0, 0, 0, 0, 1, 0, 1, 0], [44, 1, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 1], [44, 0, 0, 1, 0, 0, 0, 0, 0, 0, 2, 0, 1, 0], [44, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0], [44, 0, 0, 0, 2, 0, 0, 0, 0, 0, 1, 0, 1, 0], [44, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1], [44, 0, 0, 0, 6, 0, 0, 0, 0, 0, 1, 0, 1, 0], [44, 0, 0, 0, 7, 0, 0, 1, 0, 0, 0, 0, 0, 0], [44, 1, 0

In [55]:
data = []
data_new_test = []

for i in data_orig:
    for j in range(len(i)):
        if j % 2 != 0:
            data.append(i[j])
        else:
            data_new_test.append(i[j])



In [56]:
data_test = []

for j in range(0,len(data)):
    for i in data[j]:
        data_test.append(i)

print(len(data_test))

33768


In [57]:
val = len(data_test)/14

new_data = np.array(data_test).reshape(int(val),14)


a = "t"
b = "stay_sv"
c = "stay_ov"
d = "stay_yv"
e = "stay_mv"
f = "stay_kv"
g = "stay_pv"

h = "switch_sv"
i = "switch_ov"
j = "switch_yv"
k = "switch_mv"
l = "switch_kv"
m = "switch_pv"
n = "c"

l = [a,b,c,d,e,f,g,h,i,j,k,l,m,n]

df = pd.DataFrame(new_data, columns = l)


indiv_traces = {}



#c_data = df.loc[df['t'] == 0]
c_data = df
c_data = c_data.reset_index(drop=True)

stay_sv = c_data["stay_sv"]
stay_ov = c_data["stay_ov"]
stay_yv = c_data["stay_yv"]
stay_mv = c_data["stay_mv"]
stay_kv = c_data["stay_kv"]
stay_pv = c_data["stay_pv"]


switch_sv = c_data["switch_sv"]
switch_ov = c_data["switch_ov"]
switch_yv = c_data["switch_yv"]
switch_mv = c_data["switch_mv"]
switch_kv = c_data["switch_kv"]
switch_pv = c_data["switch_pv"]

choice = c_data["c"]

print(c_data)

n_testers = len(c_data['t'].unique())
#n_testers = 160
testers_idx = c_data['t'].values
n_tests = len(c_data['c'])
print(n_testers)

        t  stay_sv  stay_ov  stay_yv  stay_mv  stay_kv  stay_pv  switch_sv  \
0       0        0        0        0        5        0        0          0   
1       0        0        0        0        1        0        0          0   
2       0        0        0        0        2        0        0          1   
3       0        0        0        0        6        0        0          0   
4       0        0        0        0        3        0        0          0   
5       0        0        0        0        4        0        0          0   
6       0        0        0        0        3        0        0          1   
7       0        0        0        0        0        1        0          0   
8       0        0        0        0        1        0        0          0   
9       0        0        1        0        0        0        0          0   
10      0        1        0        0        0        0        0          0   
11      0        0        1        0        0        0        0 

[2412 rows x 14 columns]
201


In [58]:
print('hello')
#switch = (B0 * stay_sv) + (B1 * stay_ov) + (B2 * stay_yv) + (B3 * stay_mv) + (B4 * stay_kv) + (B5 * stay_pv)
#stay = (B0 * switch_sv) + (B1 * switch_ov) + (B2 * switch_yv) + (B3 * switch_mv) + (B4 * switch_kv) + (B5 * switch_pv)

hello


In [59]:
test_data_ids = []
test_tester_ids = []
test_level_ids = []
test_contexts = []
test_left_obstacles = []
test_right_obstacles = []
test_decisions = []
test_start_lane = []

In [60]:
data_new_test_formatted = []

for i in range(0,n_testers):
    data_new_test_formatted.append([])

    
for j in data_new_test:
    data_new_test_formatted[j[0]].append(j)
    
print(data_new_test_formatted)

[[[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1], [0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 1, 0, 1, 1], [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 6, 0, 0, 1], [0, 0, 0, 0, 4, 0, 0, 1, 0, 0, 0, 0, 0, 1], [0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1], [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 2, 0, 1, 1], [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1], [0, 0, 0, 0, 7, 0, 0, 1, 0, 0, 0, 0, 0, 1], [0, 0, 0, 0, 6, 0, 0, 0, 0, 0, 1, 0, 1, 1], [0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0], [0, 0, 0, 0, 4, 0, 0, 0, 0, 0, 1, 0, 0, 1], [0, 0, 0, 0, 7, 0, 0, 0, 0, 0, 1, 0, 1, 1]], [[1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 4, 0, 0, 0], [1, 0, 0, 0, 6, 0, 0, 0, 0, 0, 1, 0, 0, 1], [1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1], [1, 0, 0, 0, 5, 0, 0, 0, 0, 0, 1, 0, 1, 1], [1, 0, 0, 0, 2, 0, 0, 0, 0, 0, 1, 0, 1, 1], [1, 0, 0, 0, 3, 0, 0, 1, 0, 0, 0, 0, 0, 1], [1, 0, 0, 0, 7, 0, 0, 0, 0, 0, 1, 0, 1, 1], [1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 5, 0, 0, 0], [1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0], [1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1], [1, 0, 1, 0, 0, 0, 0, 0, 0, 

In [61]:

j = 0
truth_decisions = []
predictions = []
contexts = []
left = []
right = []
for j in range(0,n_testers):
    self_weight = 1
    old_weight = 1
    young_weight = 1
    man_weight = 1
    kneeling_weight = 1 
    pavement_weight = 1


    for i in data_new_test_formatted[j]:
        stay_matrix = [i[1],i[2],i[3],i[4],i[5],i[6]]
        switch_matrix = [i[7],i[8],i[9],i[10],i[11],i[12]]
        
        print(stay_matrix)
        print(switch_matrix)
        print(i[13])
        print("#########")
        
        stay_utility = self_weight*switch_matrix[0] + old_weight*switch_matrix[1] + young_weight*switch_matrix[2] + man_weight*switch_matrix[3] + kneeling_weight*switch_matrix[4] + pavement_weight*switch_matrix[5]
        switch_utility = self_weight*stay_matrix[0] + old_weight*stay_matrix[1] + young_weight*stay_matrix[2] + man_weight*stay_matrix[3] + kneeling_weight*stay_matrix[4] + pavement_weight*stay_matrix[5]
        
        overall_utility1 = switch_utility - stay_utility
        overall_utility2 = stay_utility - switch_utility
        
        probability1 = 1/(1 + np.exp(-overall_utility1))
        probability2 = 1/(1 + np.exp(-overall_utility2))
                         
        if probability1 > probability2:
            predictions.append(1)
        else:
            predictions.append(0)

        truth_decisions.append(i[13])
        #contexts.append(i[5])
        #left.append(left_matrix)
        #right.append(right_matrix)

[0, 0, 1, 0, 0, 0]
[0, 0, 0, 0, 1, 0]
1
#########
[0, 0, 0, 2, 0, 0]
[0, 0, 0, 1, 0, 1]
1
#########
[1, 0, 0, 0, 0, 0]
[0, 0, 0, 6, 0, 0]
1
#########
[0, 0, 0, 4, 0, 0]
[1, 0, 0, 0, 0, 0]
1
#########
[0, 0, 2, 0, 0, 0]
[0, 0, 0, 1, 0, 1]
1
#########
[0, 0, 1, 0, 0, 0]
[0, 0, 0, 2, 0, 1]
1
#########
[0, 0, 1, 0, 0, 0]
[0, 0, 0, 1, 0, 1]
1
#########
[0, 0, 0, 7, 0, 0]
[1, 0, 0, 0, 0, 0]
1
#########
[0, 0, 0, 6, 0, 0]
[0, 0, 0, 1, 0, 1]
1
#########
[0, 0, 0, 1, 0, 0]
[0, 0, 1, 0, 0, 0]
0
#########
[0, 0, 0, 4, 0, 0]
[0, 0, 0, 1, 0, 0]
1
#########
[0, 0, 0, 7, 0, 0]
[0, 0, 0, 1, 0, 1]
1
#########
[1, 0, 0, 0, 0, 0]
[0, 0, 0, 4, 0, 0]
0
#########
[0, 0, 0, 6, 0, 0]
[0, 0, 0, 1, 0, 0]
1
#########
[0, 0, 1, 0, 0, 0]
[0, 0, 0, 1, 0, 1]
1
#########
[0, 0, 0, 5, 0, 0]
[0, 0, 0, 1, 0, 1]
1
#########
[0, 0, 0, 2, 0, 0]
[0, 0, 0, 1, 0, 1]
1
#########
[0, 0, 0, 3, 0, 0]
[1, 0, 0, 0, 0, 0]
1
#########
[0, 0, 0, 7, 0, 0]
[0, 0, 0, 1, 0, 1]
1
#########
[1, 0, 0, 0, 0, 0]
[0, 0, 0, 5, 0, 0]
0
#########


[0, 0, 0, 3, 0, 0]
0
#########
[0, 0, 0, 2, 0, 0]
[1, 0, 0, 0, 0, 0]
1
#########
[0, 0, 0, 0, 1, 0]
[0, 1, 0, 0, 0, 0]
1
#########
[0, 0, 0, 4, 0, 0]
[1, 0, 0, 0, 0, 0]
1
#########
[0, 0, 0, 5, 0, 0]
[1, 0, 0, 0, 0, 0]
0
#########
[0, 0, 0, 6, 0, 0]
[0, 0, 0, 1, 0, 0]
0
#########
[0, 0, 0, 3, 0, 0]
[0, 0, 0, 1, 0, 1]
1
#########
[0, 0, 1, 0, 0, 0]
[0, 0, 0, 0, 1, 0]
0
#########
[0, 0, 0, 3, 0, 0]
[1, 0, 0, 0, 0, 0]
1
#########
[0, 0, 0, 1, 0, 0]
[0, 0, 0, 0, 1, 0]
1
#########
[0, 1, 0, 0, 0, 0]
[0, 0, 0, 0, 1, 0]
1
#########
[0, 0, 0, 6, 0, 0]
[0, 0, 0, 1, 0, 1]
1
#########
[0, 0, 2, 0, 0, 0]
[0, 0, 0, 1, 0, 1]
1
#########
[0, 0, 1, 0, 0, 0]
[0, 0, 0, 1, 0, 0]
1
#########
[0, 0, 0, 1, 0, 0]
[0, 0, 0, 4, 0, 0]
0
#########
[0, 0, 0, 5, 0, 0]
[0, 0, 0, 1, 0, 1]
0
#########
[0, 0, 0, 4, 0, 0]
[0, 0, 0, 1, 0, 1]
1
#########
[0, 0, 0, 1, 0, 0]
[0, 0, 0, 0, 1, 0]
0
#########
[0, 1, 0, 0, 0, 0]
[0, 0, 0, 1, 0, 0]
1
#########
[0, 0, 0, 2, 0, 0]
[0, 0, 0, 1, 0, 0]
1
#########
[0, 0, 0, 1, 0, 0]


[0, 0, 0, 6, 0, 0]
[0, 0, 0, 1, 0, 0]
1
#########
[0, 0, 0, 2, 0, 0]
[0, 0, 0, 1, 0, 1]
1
#########
[0, 0, 0, 7, 0, 0]
[1, 0, 0, 0, 0, 0]
0
#########
[0, 0, 0, 4, 0, 0]
[1, 0, 0, 0, 0, 0]
1
#########
[0, 1, 0, 0, 0, 0]
[0, 0, 0, 1, 0, 0]
0
#########
[0, 0, 0, 5, 0, 0]
[0, 0, 0, 1, 0, 1]
1
#########
[0, 0, 0, 3, 0, 0]
[0, 0, 0, 1, 0, 1]
1
#########
[1, 0, 0, 0, 0, 0]
[0, 0, 0, 2, 0, 0]
0
#########
[0, 0, 0, 7, 0, 0]
[0, 0, 0, 1, 0, 1]
1
#########
[0, 0, 0, 1, 0, 0]
[0, 0, 0, 2, 0, 0]
0
#########
[0, 0, 0, 5, 0, 0]
[1, 0, 0, 0, 0, 0]
1
#########
[0, 0, 0, 1, 0, 0]
[0, 1, 0, 0, 0, 0]
1
#########
[0, 0, 0, 3, 0, 0]
[0, 0, 0, 1, 0, 1]
1
#########
[1, 0, 0, 0, 0, 0]
[0, 0, 0, 2, 0, 0]
0
#########
[0, 0, 0, 0, 1, 0]
[0, 0, 0, 1, 0, 0]
1
#########
[0, 0, 0, 4, 0, 0]
[1, 0, 0, 0, 0, 0]
1
#########
[0, 0, 1, 0, 0, 0]
[0, 0, 0, 1, 0, 1]
1
#########
[0, 0, 0, 2, 0, 0]
[0, 0, 0, 1, 0, 1]
1
#########
[0, 0, 0, 0, 1, 0]
[0, 1, 0, 0, 0, 0]
1
#########
[0, 0, 0, 6, 0, 0]
[1, 0, 0, 0, 0, 0]
1
#########


[0, 0, 0, 2, 0, 0]
[0, 0, 0, 1, 0, 1]
1
#########
[0, 0, 0, 1, 0, 0]
[0, 0, 0, 6, 0, 0]
0
#########
[0, 0, 0, 5, 0, 0]
[1, 0, 0, 0, 0, 0]
1
#########
[0, 1, 0, 0, 0, 0]
[0, 0, 0, 0, 1, 0]
0
#########
[1, 0, 0, 0, 0, 0]
[0, 0, 0, 3, 0, 0]
0
#########
[0, 0, 0, 6, 0, 0]
[0, 0, 0, 1, 0, 1]
1
#########
[0, 0, 0, 7, 0, 0]
[0, 0, 0, 1, 0, 1]
1
#########
[0, 0, 0, 1, 0, 0]
[0, 0, 0, 6, 0, 0]
0
#########
[0, 0, 0, 5, 0, 0]
[1, 0, 0, 0, 0, 0]
0
#########
[0, 0, 0, 2, 0, 0]
[0, 0, 0, 1, 0, 0]
1
#########
[0, 0, 0, 5, 0, 0]
[0, 0, 0, 1, 0, 1]
1
#########
[0, 0, 0, 0, 1, 0]
[0, 0, 0, 1, 0, 0]
0
#########
[0, 0, 0, 2, 0, 0]
[0, 0, 0, 1, 0, 1]
1
#########
[0, 0, 0, 4, 0, 0]
[0, 0, 0, 1, 0, 0]
1
#########
[0, 0, 0, 1, 0, 0]
[0, 0, 1, 0, 0, 0]
0
#########
[0, 0, 0, 3, 0, 0]
[0, 0, 0, 1, 0, 1]
1
#########
[0, 0, 0, 6, 0, 0]
[1, 0, 0, 0, 0, 0]
1
#########
[0, 0, 0, 0, 1, 0]
[0, 0, 1, 0, 0, 0]
0
#########
[0, 0, 0, 7, 0, 0]
[0, 0, 0, 1, 0, 1]
1
#########
[0, 0, 0, 7, 0, 0]
[1, 0, 0, 0, 0, 0]
1
#########


[0, 0, 0, 1, 0, 0]
1
#########
[0, 0, 1, 0, 0, 0]
[0, 0, 0, 2, 0, 1]
1
#########
[0, 0, 0, 3, 0, 0]
[0, 0, 0, 1, 0, 1]
1
#########
[0, 0, 0, 5, 0, 0]
[1, 0, 0, 0, 0, 0]
0
#########
[0, 0, 0, 7, 0, 0]
[0, 0, 0, 1, 0, 1]
1
#########
[0, 0, 2, 0, 0, 0]
[0, 0, 0, 1, 0, 1]
1
#########
[0, 0, 0, 0, 1, 0]
[0, 0, 1, 0, 0, 0]
1
#########
[0, 0, 1, 0, 0, 0]
[0, 0, 0, 1, 0, 1]
0
#########
[0, 0, 0, 4, 0, 0]
[0, 0, 0, 1, 0, 1]
1
#########
[0, 0, 1, 0, 0, 0]
[0, 0, 0, 1, 0, 0]
1
#########
[0, 0, 0, 2, 0, 0]
[0, 0, 0, 1, 0, 1]
1
#########
[0, 0, 0, 0, 1, 0]
[0, 1, 0, 0, 0, 0]
1
#########
[0, 0, 0, 2, 0, 0]
[0, 0, 0, 1, 0, 0]
1
#########
[1, 0, 0, 0, 0, 0]
[0, 0, 0, 7, 0, 0]
1
#########
[0, 0, 1, 0, 0, 0]
[0, 0, 0, 2, 0, 1]
0
#########
[1, 0, 0, 0, 0, 0]
[0, 0, 0, 7, 0, 0]
0
#########
[0, 0, 0, 3, 0, 0]
[1, 0, 0, 0, 0, 0]
0
#########
[0, 0, 0, 1, 0, 0]
[0, 0, 0, 6, 0, 0]
0
#########
[0, 0, 0, 6, 0, 0]
[0, 0, 0, 1, 0, 1]
1
#########
[0, 0, 0, 0, 1, 0]
[0, 0, 0, 1, 0, 0]
1
#########
[0, 1, 0, 0, 0, 0]


[0, 0, 0, 1, 0, 1]
1
#########
[0, 0, 0, 2, 0, 0]
[0, 0, 0, 1, 0, 1]
1
#########
[0, 0, 0, 1, 0, 0]
[0, 0, 0, 4, 0, 0]
0
#########
[1, 0, 0, 0, 0, 0]
[0, 0, 0, 2, 0, 0]
1
#########
[0, 0, 0, 1, 0, 0]
[0, 0, 0, 0, 1, 0]
0
#########
[0, 1, 0, 0, 0, 0]
[0, 0, 0, 1, 0, 0]
0
#########
[0, 0, 0, 7, 0, 0]
[0, 0, 0, 1, 0, 1]
1
#########
[1, 0, 0, 0, 0, 0]
[0, 0, 0, 3, 0, 0]
1
#########
[0, 0, 0, 1, 0, 0]
[0, 1, 0, 0, 0, 0]
1
#########
[1, 0, 0, 0, 0, 0]
[0, 0, 0, 7, 0, 0]
0
#########
[1, 0, 0, 0, 0, 0]
[0, 0, 0, 2, 0, 0]
1
#########
[0, 0, 0, 2, 0, 0]
[0, 0, 0, 1, 0, 0]
1
#########
[0, 0, 1, 0, 0, 0]
[0, 0, 0, 0, 1, 0]
1
#########
[0, 1, 0, 0, 0, 0]
[0, 0, 1, 0, 0, 0]
0
#########
[0, 0, 0, 1, 0, 0]
[0, 0, 0, 0, 1, 0]
0
#########
[0, 0, 1, 0, 0, 0]
[0, 0, 0, 1, 0, 1]
1
#########
[0, 0, 0, 6, 0, 0]
[0, 0, 0, 1, 0, 1]
0
#########
[0, 0, 0, 1, 0, 0]
[0, 0, 0, 6, 0, 0]
0
#########
[0, 1, 0, 0, 0, 0]
[0, 0, 0, 1, 0, 0]
1
#########
[0, 0, 0, 1, 0, 0]
[0, 0, 1, 0, 0, 0]
0
#########
[0, 0, 0, 1, 0, 0]


1
#########
[0, 0, 0, 2, 0, 0]
[0, 0, 0, 1, 0, 1]
1
#########
[0, 0, 1, 0, 0, 0]
[0, 1, 0, 0, 0, 0]
1
#########
[0, 1, 0, 0, 0, 0]
[0, 0, 0, 0, 1, 0]
1
#########
[0, 0, 0, 7, 0, 0]
[0, 0, 0, 1, 0, 1]
1
#########
[0, 0, 0, 1, 0, 0]
[0, 0, 0, 2, 0, 0]
0
#########
[0, 0, 0, 0, 1, 0]
[0, 0, 1, 0, 0, 0]
0
#########
[0, 0, 0, 7, 0, 0]
[1, 0, 0, 0, 0, 0]
0
#########
[0, 0, 2, 0, 0, 0]
[0, 0, 0, 1, 0, 1]
1
#########
[0, 0, 0, 6, 0, 0]
[0, 0, 0, 1, 0, 1]
0
#########
[1, 0, 0, 0, 0, 0]
[0, 0, 0, 5, 0, 0]
1
#########
[0, 0, 1, 0, 0, 0]
[0, 0, 0, 1, 0, 1]
1
#########
[0, 0, 0, 6, 0, 0]
[0, 0, 0, 1, 0, 0]
1
#########
[0, 1, 0, 0, 0, 0]
[0, 0, 0, 0, 1, 0]
1
#########
[0, 0, 0, 0, 1, 0]
[0, 0, 0, 1, 0, 0]
0
#########
[0, 0, 0, 4, 0, 0]
[0, 0, 0, 1, 0, 1]
0
#########
[0, 0, 0, 4, 0, 0]
[0, 0, 0, 1, 0, 1]
1
#########
[0, 0, 0, 2, 0, 0]
[0, 0, 0, 1, 0, 0]
1
#########
[1, 0, 0, 0, 0, 0]
[0, 0, 0, 3, 0, 0]
0
#########
[0, 0, 0, 1, 0, 0]
[0, 0, 0, 6, 0, 0]
0
#########
[0, 0, 0, 2, 0, 0]
[0, 0, 0, 1, 0, 1]


#########
[0, 0, 1, 0, 0, 0]
[0, 1, 0, 0, 0, 0]
1
#########
[0, 1, 0, 0, 0, 0]
[0, 0, 0, 1, 0, 0]
0
#########
[0, 0, 0, 2, 0, 0]
[0, 0, 0, 1, 0, 1]
1
#########
[0, 0, 0, 4, 0, 0]
[0, 0, 0, 1, 0, 0]
1
#########
[1, 0, 0, 0, 0, 0]
[0, 0, 0, 4, 0, 0]
0
#########
[0, 0, 0, 6, 0, 0]
[0, 0, 0, 1, 0, 1]
1
#########
[0, 0, 0, 7, 0, 0]
[0, 0, 0, 1, 0, 1]
1
#########
[0, 0, 0, 1, 0, 0]
[0, 0, 0, 2, 0, 0]
0
#########
[0, 0, 2, 0, 0, 0]
[0, 0, 0, 1, 0, 1]
1
#########
[0, 0, 1, 0, 0, 0]
[0, 0, 0, 2, 0, 1]
1
#########
[0, 0, 0, 4, 0, 0]
[0, 0, 0, 1, 0, 1]
1
#########
[0, 0, 0, 6, 0, 0]
[1, 0, 0, 0, 0, 0]
1
#########
[0, 0, 0, 1, 0, 0]
[0, 0, 0, 2, 0, 0]
0
#########
[0, 0, 0, 7, 0, 0]
[0, 0, 0, 1, 0, 1]
1
#########
[0, 0, 0, 2, 0, 0]
[0, 0, 0, 1, 0, 1]
1
#########
[0, 1, 0, 0, 0, 0]
[0, 0, 0, 1, 0, 0]
0
#########
[0, 0, 0, 6, 0, 0]
[1, 0, 0, 0, 0, 0]
1
#########
[0, 0, 0, 2, 0, 0]
[1, 0, 0, 0, 0, 0]
0
#########
[1, 0, 0, 0, 0, 0]
[0, 0, 0, 7, 0, 0]
1
#########
[0, 0, 0, 6, 0, 0]
[0, 0, 0, 1, 0, 1]
1


In [62]:
i = 0
total = 0
correct = 0
for j in truth_decisions:
    print(j)
    if j == predictions[i]:
        correct = correct + 1
    total = total + 1
    i = i + 1
        

1
1
1
1
1
1
1
1
1
0
1
1
0
1
1
1
1
1
1
0
0
1
1
0
1
0
0
1
1
1
1
1
1
0
0
1
1
1
1
1
0
1
1
0
0
0
1
0
1
1
0
0
0
1
1
1
0
1
1
1
1
0
0
1
1
1
1
1
1
0
0
1
1
0
1
1
1
1
1
1
1
0
1
1
1
0
1
0
0
0
0
1
1
0
1
1
1
1
1
0
1
1
0
1
1
1
0
0
1
0
0
0
0
1
0
1
1
1
1
1
0
1
0
0
0
1
0
1
1
1
1
1
0
1
1
1
1
1
1
0
0
1
0
1
0
0
1
1
0
0
1
0
1
1
0
0
1
1
1
1
1
1
0
0
1
1
0
1
1
0
0
1
0
1
1
1
1
1
1
1
1
0
0
1
1
1
1
1
1
0
1
0
1
1
1
1
0
1
0
1
1
1
1
1
1
0
1
0
0
1
0
1
1
0
1
0
0
1
1
1
1
1
1
1
0
0
1
1
1
0
0
1
0
1
1
1
1
1
1
0
0
1
0
1
1
0
1
0
1
1
0
1
0
0
0
0
0
1
0
1
1
1
1
1
1
1
0
1
1
1
1
1
1
1
0
0
0
0
1
1
1
1
0
1
0
0
0
1
1
0
1
0
1
0
0
1
1
1
0
1
1
1
1
1
0
1
0
0
1
1
1
1
1
1
0
0
0
1
0
1
1
1
1
1
0
0
1
1
1
1
1
1
0
1
1
1
1
0
0
0
1
0
1
0
0
1
1
0
0
1
1
1
1
0
1
1
1
0
1
0
0
1
1
1
1
1
0
0
0
1
1
1
1
0
1
1
0
0
0
1
1
1
0
1
1
1
0
1
1
1
1
0
1
1
1
1
1
1
1
1
1
1
0
0
1
1
0
1
0
1
0
0
0
0
0
0
1
1
1
1
1
0
1
1
0
1
1
1
0
1
1
1
0
1
1
0
1
1
1
0
1
1
1
1
0
1
1
1
0
1
1
1
0
1
0
0
0
1
0
0
1
1
1
1
0
1
1
0
1
1
1
0
1
0
1
1
0
1
0
1
1
1
0
1
1
1
1
1
1
1
1
0
0
0
1
1
0
0
0
1


In [63]:
print(correct/total)

0.6227197346600332


In [64]:
print(self_weight)
print(old_weight)
print(young_weight)
print(man_weight)
print(kneeling_weight)
print(pavement_weight)

1
1
1
1
1
1
